In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

def calculate_alpha_beta(tickers, weights, benchmark, start_date, end_date):
    # Fetch data from Yahoo Finance API
    portf_returns = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)['Adj Close'].pct_change()
        print(f'YF data size: {ticker}: {data.size}')
        print(f'Volatility of {ticker}: {np.std(data) * np.sqrt(252):.4f}')
        portf_returns[ticker] = data

    benchmark_returns = yf.download(benchmark, start=start_date, end=end_date)['Adj Close'].pct_change()
    print(f'Volatility of {benchmark}: {np.std(benchmark_returns) * np.sqrt(252):.4f}')

    # Omit the first row which may contain NaN values
    portf_returns = portf_returns.iloc[1:]
    benchmark_returns = benchmark_returns.iloc[1:]

    # Calculate portfolio returns
    portf_returns['Portfolio'] = np.dot(portf_returns, weights)

    # Calculate covariance of portfolio and benchmark returns
    cov_matrix = np.cov(portf_returns['Portfolio'], benchmark_returns)

    # Calculate beta
    beta = cov_matrix[0, 1] / benchmark_returns.var()

    # Calculate alpha
    alpha = (portf_returns['Portfolio'].mean() - beta * benchmark_returns.mean())

    portf_stdev = portf_returns['Portfolio'].std()

    return alpha, beta, portf_returns['Portfolio'].mean(), portf_stdev

In [ ]:
# tickers = ['ADBE', 'AMAT', 'ARKK', 'BLDR', 'BRK-B', 'ETHE', 'FTNT', 'GBTC', 'QQQE', 'VXX', 'TMF', 'TSLA']
# weights = [0.075, 0.075, 0.1, 0.1, 0.125, 0.05, 0.075, 0.05, 0.125, -0.125, 0.1, 0.075]
# tickers = ["ACLS", "AMPH", "AMR", "ANF", "APP", "ARCH", "ASC", "BXC", "CAAP", "CAH", "CLS", "CMCSA", "COP", "CRM", "CVX", "DINO", "GOOGL", "GRBK", "HLIT", "JXN", "META", "MFC", "MHO", "MOD", "MPC", "NUE", "OKTA", "PEP", "PERI", "POWL", "SMCI", "STRL", "SU", "TEX", "TGLS", "TMUS", "TWLO", "UBER", "URI", "VLO", "XOM"]
tickers = ["AAPL","ACLS","ACN","AGX","ALL","AMAT","AMC","AMD","AMPH","AMZN","ANET","ANF","APP","ARM","ATGE","AVGO","AXP","BIL","BKR","BLBD","BLDR","BRK-B","BTC","CAAP","CCL","CDNS","CLF","CLS","CMCSA","CRH","CRM","CRMD","CRWD","DELL","DFS","DNA","DOLE","EA","EAT","EB","EEM","EMB","ENPH","ENVX","EPAM","EPP","ETH","ETHE","FSLY","GBTC","GD","GDRX","GEHC","GLD","GM","GOOGL","GPRO","GRBK","HMY","HON","HUM","HYG","IAG","IBKR","IEF","INGR","INTA","INTC","INTU","KEX","LAZR","LC","LCID","LHX","LQD","LVS","MAXN","MCD","MDGL","MDY","MELI","META","MFC","MHO","MIRM","MO","MOD","MSFT","MTCH","NKLA","NOW","NVDA","OKTA","PANW","PEP","PGR","PLTR","PM","POWL","PPC","PRCT","PSIX","PYCR","PYPL","QTWO","RCL","RGA","RGP","RNA","RSI","RTX","SCHYY","SEDG","SFM","SHOP","SHY","SKYW","SLVM","SMCI","SMH","SQ","STRL","SVIX","SYF","TCOM","TEX","TIP","TLT","TMF","TMUS","TMV","TQQQ","TSLA","TSLL","TSM","TTD","TWLO","UAL","UBER","UGL","UNG","UPRO","URI","USO","UWM","VIST","VKTX","VNQ","VWO","VXX","VXZ","WFC","WGS","WYNN","ZETA","ZI","ZIM"]
nTickers = len(tickers)
weights = [1.0 / nTickers] * nTickers # equal weight
benchmark = 'SPY'
start_date = '2024-05-21'
end_date = '2024-11-21'

alpha, beta, portfolio_return, portfolio_volatility = calculate_alpha_beta(tickers, weights, benchmark, start_date, end_date)

print(f'Alpha: {alpha*252:.4f}')
print(f'Beta: {beta:.4f}')
print(f'Portfolio Return: {portfolio_return*252:.4f}')
print(f'Portfolio Volatility: {portfolio_volatility*np.sqrt(252):.4f}')

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close'].pct_change()
    return data

def calculate_alpha_beta_for_ticker(ticker_data, benchmark_data):
    # Align and clean data
    data = pd.concat([ticker_data, benchmark_data], axis=1).dropna()
    ticker_returns = data.iloc[:, 0]
    benchmark_returns = data.iloc[:, 1]

    # Calculate covariance and beta
    cov_matrix = np.cov(ticker_returns, benchmark_returns)
    beta = cov_matrix[0, 1] / benchmark_returns.var()

    # Calculate alpha
    alpha = ticker_returns.mean() - beta * benchmark_returns.mean()

    # Return alpha, beta, and standard deviation
    return alpha, beta

def calculate_portfolio_alpha_beta(ticker_data, weights, benchmark_data):
    # Replace NaNs with 0
    ticker_data = ticker_data.fillna(0)

    # Align and clean data
    data = pd.concat([ticker_data, benchmark_data], axis=1).dropna()
    ticker_returns = data.iloc[:, :-1]  # All tickers
    benchmark_returns = data.iloc[:, -1]  # Benchmark

    # Calculate portfolio returns
    portfolio_returns = np.dot(ticker_returns.values, weights)

    # Calculate covariance and beta
    cov_matrix = np.cov(portfolio_returns, benchmark_returns)
    beta = cov_matrix[0, 1] / benchmark_returns.var()

    # Calculate alpha
    alpha = portfolio_returns.mean() - beta * benchmark_returns.mean()

    return alpha, beta

def calculate_and_export_alpha_beta(tickers, benchmark, start_date, end_date, output_file):
    # Fetch benchmark data
    benchmark_data = fetch_data(benchmark, start_date, end_date)

    # Fetch ticker data
    ticker_data = fetch_data(tickers, start_date, end_date)

    # Log tickers' data length
    ticker_lengths = ticker_data.notna().sum()
    print("Tickers' data length:")
    for ticker, length in ticker_lengths.items():
        print(f"{ticker}: {length} day")

    results = []
    
    for ticker in tickers:
        alpha, beta = calculate_alpha_beta_for_ticker(ticker_data[ticker], benchmark_data)
        results.append({
            "Ticker": ticker,
            "Alpha (annualized)": alpha * 252,  # Annualized alpha
            "Beta": beta
        })
        print(f"Ticker: {ticker}, Alpha: {alpha * 252:.4f}, Beta: {beta:.4f}")

    # Calculate portfolio alpha and beta
    portfolio_alpha, portfolio_beta = calculate_portfolio_alpha_beta(ticker_data, weights, benchmark_data)
    results.append({
        "Ticker": "Portfolio",
        "Alpha (annualized)": portfolio_alpha * 252,
        "Beta": portfolio_beta
    })
    print(f"Portfolio Alpha: {portfolio_alpha * 252:.4f}, Beta: {portfolio_beta:.4f}")

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Export to CSV
    results_df.to_csv(output_file, index=False)
    print(f"Results are exported: {output_file}")


tickers = ["AAPL","ACLS","ACN","AGX","ALL","AMAT","AMC","AMD","AMPH","AMZN","ANET","ANF","APP","ARM","ATGE","AVGO","AXP","BIL","BKR","BLBD","BLDR","BRK-B","BTC","CAAP","CCL","CDNS","CLF","CLS","CMCSA","CRH","CRM","CRMD","CRWD","DELL","DFS","DNA","DOLE","EA","EAT","EB","EEM","EMB","ENPH","ENVX","EPAM","EPP","ETH","ETHE","FSLY","GBTC","GD","GDRX","GEHC","GLD","GM","GOOGL","GPRO","GRBK","HMY","HON","HUM","HYG","IAG","IBKR","IEF","INGR","INTA","INTC","INTU","KEX","LAZR","LC","LCID","LHX","LQD","LVS","MAXN","MCD","MDGL","MDY","MELI","META","MFC","MHO","MIRM","MO","MOD","MSFT","MTCH","NKLA","NOW","NVDA","OKTA","PANW","PEP","PGR","PLTR","PM","POWL","PPC","PRCT","PSIX","PYCR","PYPL","QTWO","RCL","RGA","RGP","RNA","RSI","RTX","SCHYY","SEDG","SFM","SHOP","SHY","SKYW","SLVM","SMCI","SMH","SQ","STRL","SVIX","SYF","TCOM","TEX","TIP","TLT","TMF","TMUS","TMV","TQQQ","TSLA","TSLL","TSM","TTD","TWLO","UAL","UBER","UGL","UNG","UPRO","URI","USO","UWM","VIST","VKTX","VNQ","VWO","VXX","VXZ","WFC","WGS","WYNN","ZETA","ZI","ZIM"]
nTickers = len(tickers)
# weights = [1.0 / nTickers] * nTickers # equal weight
weights = [0.004591022,0.000141616,0.004269127,0.006874987,0.002550157,-0.002342739,-0.002381179,0.002892118,0.00032016,0.008864375,0.014098031,0.005149211,0.025376731,0.002813057,0.00107331,0.011913147,0.00607917,0.008974269,0.000793904,0.004324563,0.001645025,0.012420972,0.021109651,0.000855792,0.010352568,0.00651701,-0.001918736,0.022141851,0.000432723,0.000785273,0.000816195,0.002255268,0.006488268,0.010821106,0.00249699,-0.001862143,0.000624526,0.002077169,0.011049392,-0.002950224,0.007200086,0.001295856,-0.001846926,-0.001630787,-0.00279397,0.006097411,0.000628485,0.012455364,-0.002351904,0.028049951,0.000939747,-0.001510928,0.003659639,0.011777716,0.006423578,0.012627768,-0.002021466,0.004361936,0.000793815,0.001254479,-0.002627305,0.002265991,0.0090988,0.001477648,0.001374117,0.000640498,0.001027573,0.002235959,-0.002249485,0.00494504,-0.001474356,0.006251218,-0.001956153,0.001485167,0.005064722,0.001084166,-0.001514353,-0.005104052,0.000642189,0.005871706,0.006959788,0.012528463,0.005757319,0.011165603,-0.002514253,0.000925421,0.004455501,0.00868641,0.000580524,-0.000895256,0.015839557,0.016643828,0.000436994,0.006574937,0.000552805,0.003542004,0.00875942,0.000732818,0.015016377,0.007216414,0.002517857,0.006907288,-0.002661208,0.010014611,0.006482306,0.007968452,0.005607161,-0.002234357,0.00593177,0.002857281,0.002013636,0.000366476,-0.00162309,0.001681374,0.006014524,0.004471607,0.010582677,0.000728457,0.002160457,0.003807662,0.006720513,0.007609896,0.004100104,0.010488133,0.007303261,0.000314643,0.005035268,0.004822955,0.00056642,0.009336206,-0.006052742,0.009216969,0.0322765,0.005576551,0.008205814,0.006280404,0.001230276,0.00104666,0.005269871,0.001124742,-0.003850774,0.001912863,0.001115622,-0.002483287,0.000794527,0.000812769,0.000116389,0.022657373,0.001360413,0.001156776,-0.000942995,0.001070997,0.006673575,0.000507024,0.00950692,-0.002271953,0.000936588]
benchmark = 'SPY'
start_date = '2024-08-21'
end_date = '2024-11-21'
output_file = 'D:\\Temp\\alphaBeta.csv'

calculate_and_export_alpha_beta(tickers, benchmark, start_date, end_date, output_file)

# print(f'Alpha: {alpha * 252:.4f}')
# print(f'Beta: {beta:.4f}')
# print(f'Portfolio Return: {portfolio_return * 252:.4f}')
# print(f'Portfolio Volatility: {portfolio_volatility * np.sqrt(252):.4f}')


In [ ]:
import yfinance as yf
import pandas as pd

tickers = ["SPY", "ACLS", "AMPH", "AMR", "ANF", "APP", "ARCH", "ASC", "BXC", "CAAP", "CAH", "CLS", "CMCSA", "COP", "CRM", "CVX", "DINO", "GOOGL", "GRBK", "HLIT", "JXN", "LTHM", "META", "MFC", "MHO", "MOD", "MPC", "NUE", "OKTA", "PEP", "PERI", "POWL", "SMCI", "STRL", "SU", "TA", "TEX", "TGLS", "TMUS", "TWLO", "UBER", "URI", "VLO", "XOM"]
start_date = "2021-01-01"
data = pd.DataFrame()

for ticker in tickers:
    tmp = yf.download(ticker, start=start_date)
    tmp = tmp[["Adj Close"]]
    tmp.columns = [ticker]
    if data.empty:
        data = tmp
    else:
        data = data.join(tmp)

data.to_csv("D:\Temp\stocks_data.csv")